In [17]:
# Importando bibliotecas numérica e matemática
import numpy as np
import math
# Importando bibliotecas para visualização
import matplotlib.pyplot as plt
# Importando bibliotecas para manipulação simbólica
from sympy import symbols, diff, lambdify, cos, sin, log
# Importando biblioteca para criar tabelas bonitas
from prettytable import PrettyTable

In [18]:
def metodo_newton_raphson(f, ponto_inicial, precisao, max_iteracoes, exibir_grafico=True):
    """
    Aplica o método de Newton-Raphson para encontrar uma raiz da função f.

    Parâmetros:
    - f: Função a ser analisada.
    - ponto_inicial: Ponto inicial para iniciar as iterações.
    - precisao: Precisão desejada para a raiz.
    - max_iteracoes: Número máximo de iterações.
    - exibir_grafico: Se True, exibe um gráfico do processo.

    Retorna:
    - A raiz encontrada.
    """
    print('\n--- Método de Newton Raphson ---\n')
    if abs(f(ponto_inicial)) < precisao:
        print('\nRESULTADO ')
        return ponto_inicial

    valores_x = []
    limite_a = ponto_inicial

    tabela = criar_tabela_newton()

    x = symbols('x')
    derivada = lambda x_temp: diff(f(x), x).subs(x, x_temp)

    ponto_proximo = calcular_proximo_ponto_newton(ponto_inicial, f, derivada)
    valores_x.append(ponto_inicial)

    for k, (ponto_atual, ponto_proximo) in enumerate(iteracoes_newton(ponto_inicial, max_iteracoes, f, derivada, precisao), start=1):
        tabela.add_row([k, ponto_atual, f(ponto_atual), derivada(ponto_atual), ponto_proximo])
        valores_x.append(ponto_proximo)

    if exibir_grafico:
        valores_y = [f(i) for i in valores_x]
        plotar_grafico(limite_a - 1, float(ponto_proximo) + 1, f, valores_x, valores_y, 'newton_raphson', ultimo_ponto=(valores_x[-1], valores_y[-1]))

    print(tabela)
    print('\nRESULTADO ')
    return ponto_proximo

def iteracoes_newton(ponto_inicial, max_iteracoes, f, derivada, precisao):
    """
    Gera iterações para o método de Newton-Raphson.

    Parâmetros:
    - ponto_inicial: Ponto inicial.
    - max_iteracoes: Número máximo de iterações.
    - f: Função.
    - derivada: Função que calcula a derivada.
    - precisao: Precisão desejada.

    Yields:
    - Tuplas (ponto_atual, ponto_proximo) representando os pontos em cada iteração.
    """
    for _ in range(max_iteracoes):
        ponto_proximo = calcular_proximo_ponto_newton(ponto_inicial, f, derivada)
        yield ponto_inicial, ponto_proximo
        if abs(ponto_proximo - ponto_inicial) < precisao:
            break
        ponto_inicial = ponto_proximo

def calcular_proximo_ponto_newton(ponto_inicial, f, derivada):
    """
    Calcula o próximo ponto de iteração usando o método de Newton-Raphson.

    Parâmetros:
    - ponto_inicial: Ponto inicial.
    - f: Função.
    - derivada: Função que calcula a derivada.

    Retorna:
    - O próximo ponto calculado.
    """
    derivada_ponto_inicial = derivada(ponto_inicial)
    if derivada_ponto_inicial == 0:
        # Evitar divisão por zero
        raise ValueError('A derivada no ponto inicial é zero. O método de Newton-Raphson não pode ser aplicado.')
    return ponto_inicial - (f(ponto_inicial) / derivada_ponto_inicial)

def criar_tabela_newton():
    """
    Cria e retorna uma tabela para exibir os resultados do método de Newton-Raphson.
    """
    return PrettyTable(['n', 'x^(x0)', 'f(x^(0))', 'f\'(x^(0))', 'x1'])


In [22]:
def metodo_secante(f, x0, x1, precisao, max_iteracoes, exibir_grafico=True):
    """
    Aplica o método da secante para encontrar uma raiz da função f.

    Parâmetros:
    - f: Função a ser analisada.
    - x0, x1: Pontos iniciais para iniciar as iterações.
    - precisao: Precisão desejada para a raiz.
    - max_iteracoes: Número máximo de iterações.
    - exibir_grafico: Se True, exibe um gráfico do processo.

    Retorna:
    - A raiz encontrada.
    """
    print('\n--- Método da Secante ---\n')

    # Verifica se os pontos iniciais já estão dentro da precisão desejada
    if abs(f(x0)) < precisao:
        print('\nRESULTADO ')
        return x0
    elif abs(f(x1)) < precisao or abs(x1 - x0) < precisao:
        print('\nRESULTADO ')
        return x1

    valores_x = [x0, x1]
    limite_a = x0
    limite_b = x1

    tabela = PrettyTable(['n', 'x_(n-1)', 'f(x_(n-1))', 'x_n', 'x_(n+1)'])

    k = 1

    while k <= max_iteracoes:
        x2 = x1 - (f(x1) / (f(x1) - f(x0))) * (x1 - x0)

        tabela.add_row([k, x0, f(x0), x1, x2])
        valores_x.append(x2)

        if abs(f(x2)) < precisao or abs(x2 - x1) < precisao:
            break

        x0, x1 = x1, x2
        k += 1

    if exibir_grafico:
        valores_y = [f(i) for i in valores_x]
        plotar_grafico(limite_a - 1, limite_b + 1, f, valores_x, valores_y, 'secante', ultimo_ponto=(valores_x[-1], valores_y[-1]))

    print(tabela)
    print('\nRESULTADO ')
    return valores_x[-1]


In [20]:
def metodo_bisseccao(f, a, b, precisao, max_iteracoes, exibir_grafico=True):
    if f(a) * f(b) >= 0:
        raise ValueError("Intervalo inválido. A função não muda de sinal no intervalo.")

    iteracao = 1
    valores_x = []
    limite_inferior = a
    limite_superior = b

    tabela = PrettyTable(['n', 'a', 'b', 'x', 'f(x)', 'f(a)', 'f(a) * f(x)'])

    while iteracao <= max_iteracoes and abs(b - a) >= precisao:
        x = (a + b) / 2
        valores_x.append(x)

        tabela.add_row([iteracao, a, b, x, f(x), f(a), f(a) * f(x)])

        if f(a) * f(x) > 0:
            a = x
        else:
            b = x

        iteracao += 1

    if exibir_grafico:
        valores_y = [f(i) for i in valores_x]
        plotar_grafico(limite_inferior, limite_superior, f, valores_x, valores_y, 'bisseccao', ultimo_ponto=(valores_x[-1], valores_y[-1]))

    print(tabela)

    print('\nRESULTADO ')
    return (a + b) / 2

def plotar_grafico(limite_inferior, limite_superior, f, pontox=None, pontosy=None, metodo=None, ultimo_ponto=None):
    x = np.linspace(limite_inferior, limite_superior, 100)
    y = f(x)

    plt.plot(x, y, label='Função', linewidth=3)

    if pontox is not None and pontosy is not None:
        plt.scatter(pontox[:-1], pontosy[:-1], color='red', label='Pontos das iterações')

    if ultimo_ponto is not None:
        plt.scatter(ultimo_ponto[0], ultimo_ponto[1], color='green', label='Solução encontrada')

    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.grid(color='gray', linestyle='--', linewidth=0.5)

    if ultimo_ponto is None:
        plt.title(f'Gráfico da Função')
    else:
        plt.title(f'Gráfico da Função - Método {metodo}')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()

    print("\n\n")
    plt.show()


In [21]:
def metodo_falsa_posicao(f, a, b, precisao, max_iteracoes, exibir_grafico=True):
    """
    Aplica o método da falsa posição para encontrar uma raiz da função f.

    Parâmetros:
    - f: Função a ser analisada.
    - a, b: Limites iniciais para iniciar as iterações.
    - precisao: Precisão desejada para a raiz.
    - max_iteracoes: Número máximo de iterações.
    - exibir_grafico: Se True, exibe um gráfico do processo.

    Retorna:
    - A raiz encontrada.
    """
    print('\n--- Método da Falsa Posição ---\n')

    # Verifica se a função muda de sinal nos limites iniciais
    if f(a) * f(b) >= 0:
        return 'Não é possível calcular'

    valores_x = []  # Lista para armazenar os pontos das iterações
    limite_a = a    # Salva limites A e B
    limite_b = b

    tabela = PrettyTable(['n', 'a', 'b', 'x', 'f(x)', 'f(a) * f(x)'])

    k = 1
    x = ((a * f(b)) - (b * f(a))) / (f(b) - f(a))

    while k <= max_iteracoes and abs(b - a) >= precisao:
        valores_x.append(x)

        tabela.add_row([k, a, b, x, f(x), f(a) * f(x)])

        # Verifica a mudança de sinal para determinar o novo intervalo
        if f(a) * f(x) > 0:
            a = x
        else:
            b = x

        x = ((a * f(b)) - (b * f(a))) / (f(b) - f(a))
        k += 1

    if exibir_grafico:
        # Calcula valores de y para plotar gráfico
        valores_y = [f(i) for i in valores_x]
        plotar_grafico(limite_a, limite_b, f, valores_x, valores_y, 'falsa_posicao', ultimo_ponto=(valores_x[-1], valores_y[-1]))

    print(tabela)
    print('\nRESULTADO ')
    return x
